In [ ]:
# !pip install langchain

# !pip install -U langchain-community

# !pip install pypdf

# !pip install -U tqdm

# !pip install -U ipywidgets  

# !pip install -U sentence-transformers==2.2.2

# !pip install torch transformers

# !pip install InstructorEmbedding
# pip install tqdm
# !pip install --upgrade sentence-transformers huggingface-hub
# pip install faiss-cpu
# pip install --quiet langchain_experimental
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

  Using cached langchain-0.3.19-py3-none-any.whl.metadata (7.9 kB)
  Using cached langchain_core-0.3.37-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.3.6-py3-none-any.whl.metadata (1.9 kB)
  Using cached langsmith-0.3.10-py3-none-any.whl.metadata (14 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached SQLAlchemy-2.0.38-cp311-cp311-win_amd64.whl.metadata (9.9 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached aiohttp-3.11.12-cp311-cp311-win_amd64.whl.metadata (8.0 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Using cached aiohappyeyeballs-2.4.6-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached fro

In [ ]:
# pip install Whoosh

Note: you may need to restart the kernel to use updated packages.


# Libraries

In [1]:
import os


os.chdir("../")


In [3]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

from langchain_experimental.text_splitter import SemanticChunker

from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from tqdm import tqdm


In [8]:
import re

In [10]:
import os
from whoosh.index import create_in, open_dir
from whoosh.fields import Schema, TEXT, ID, DATETIME, NUMERIC
import datetime
from whoosh.qparser import QueryParser


# Documents

## Load 

In [5]:
DATASET_PATH = "Dataset"

In [6]:
loader=PyPDFDirectoryLoader(path=DATASET_PATH,
                            recursive=True,
                            silent_errors=True)

In [7]:
documnets=loader.load()

## Preprocess Load DOCS

In [9]:
non_empty_documents = []
empty_documents = []

for document in documnets:
    # Check if page content has words (not just numbers)
    if re.search(r'\w+', document.page_content):
        page_content= document.page_content.replace('\n',' ')
        # Additional check: if content is only numbers and length < 3, consider empty
        if not page_content.isdigit() and len(page_content.split()) >1:
            non_empty_documents.append(document)
        else:
            empty_documents.append(document)
    else:
        empty_documents.append(document)

# Vectorize

## Embedding Model Load

In [51]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.is_available()

True

In [52]:
embedding_model=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2',
                                      model_kwargs = {'device': device})

C:\Users\User\AppData\Local\Temp\ipykernel_27088\55583320.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2',


## Split Text

### Recursive

In [63]:

text_splitter_recursive = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
    length_function=len,
    add_start_index=True
)

In [64]:
split_documents = text_splitter_recursive.split_documents(non_empty_documents)

### Sementic

In [55]:
text_splitter_semantic=SemanticChunker(embeddings=embedding_model,
                                       breakpoint_threshold_amount=0.95)

In [56]:
sementic_documnets=text_splitter_semantic.split_documents(non_empty_documents)

In [65]:
# Remove Duplicates chunks
def remove_duplicates(documents):
    unique_documents = []
    seen_texts = set()

    for doc in documents:
        if doc.page_content not in seen_texts:
            unique_documents.append(doc)
            seen_texts.add(doc.page_content)

    return unique_documents

unique_split_documents = remove_duplicates(split_documents)
unique_sementic_documnets = remove_duplicates(sementic_documnets)


### Graph Structure

## Save to FAISS

### Option 1

In [9]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [10]:
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')


tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [14]:
embeddings = []
metadatas = []

# Use tqdm to show progress
for doc in tqdm(split_documents, desc="Embedding document splits"):
    # Tokenize the split text
    encoded_input = tokenizer(doc.page_content, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    embedding = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    embedding = F.normalize(embedding, p=2, dim=1)

    # Save embedding and metadata
    embeddings.append(embedding.squeeze().cpu().numpy())
    metadatas.append(doc.metadata)

Embedding document splits: 100%|██████████| 14743/14743 [30:36<00:00,  8.03it/s] 


In [24]:
import pickle

# Save embeddings list
with open('embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

# Save metadatas list
with open('metadatas.pkl', 'wb') as f:
    pickle.dump(metadatas, f)

### Option 2

In [66]:
hybride_chunk = unique_split_documents+unique_sementic_documnets

In [67]:
# Initialize LangChain's FAISS wrapper
vector_store = FAISS.from_documents(unique_sementic_documnets,embedding_model)


In [68]:
vector_store.save_local("vector_db/faiss_index_hybride")

# Key Word Base

In [12]:
from pathlib import Path

In [13]:
KEY_WORD_BASE_INDEX_DIR="key_word_index"
if not os.path.exists(KEY_WORD_BASE_INDEX_DIR):
    os.makedirs(KEY_WORD_BASE_INDEX_DIR)

In [11]:
# Define a schema that includes your metadata fields
schema = Schema(
    title=TEXT(stored=True),          # e.g. "The Two Towers"
    author=TEXT(stored=True),         # e.g. "J. R. R. Tolkien"
    total_pages=NUMERIC(stored=True, numtype=int),  # e.g. 461
    page=NUMERIC(stored=True, numtype=int),         # e.g. 0
    page_label=TEXT(stored=True),  # e.g. "C"
    content=TEXT(stored=True)    # The main content of the document
)

In [14]:
ix = create_in(KEY_WORD_BASE_INDEX_DIR, schema)
writer = ix.writer()

In [17]:
for doc in non_empty_documents:
    
    data={
        "title": doc.metadata["title"],
        "author": doc.metadata["author"],
        "total_pages":int(doc.metadata["total_pages"]),
        "page": int(doc.metadata["page"]) ,
        "page_label": doc.metadata["page_label"],
        "content": doc.page_content 
    }
    
    writer.add_document(**data)

# Commit the writer to save the documents.
writer.commit()

print("Indexing complete!")
    

Indexing complete!


In [20]:
# Open the index for querying.
with ix.searcher() as searcher:
    # Create a query parser for the "content" field.
    parser = QueryParser("content", ix.schema)
    
    # Parse your query text. Replace "two towers" with your actual query.
    query = parser.parse("contents")
    
    # Execute the search and limit to 10 results.
    results = searcher.search(query, limit=10)
    
    # Iterate over the results and print out the document fields.
    for hit in results:
        print(hit.fields())
        print("------")

{'author': 'J. R. R. Tolkien', 'content': 'viii contents\nIV A Journey in the Dark 384\nV The Bridge of Khazad-du ˆm 418\nVI Lothlo ´rien 433\nVII The Mirror of Galadriel 459\nVIII Farewell to Lo ´rien 478\nIX The Great River 495\nX The Breaking of the Fellowship 515\nMaps                                                            533\nWorks By J.R.R. Tolkien                            540\nAbout The Publisher                                         542\nCopyright                                                    541', 'page': 6, 'page_label': 'vi', 'title': 'The Fellowship of the Ring', 'total_pages': 571}
------
{'author': 'J. R. R. Tolkien', 'content': 'xii contents\nappendices\na annals of the kings and rulers 1351\nI The Nu´meno´rean Kings 1352\nII The House of Eorl 1395\nIII Durin’s Folk 1406\nb the tale of years\n(chronology of the westlands ) 1420\nc family trees (hobbits ) 1444\nd calendars 1451\ne writing and spelling 1461\nI Pronunciation of Words and Names 1461\nII Writing